# Train model

## Описание решения

В процессе решения задачи задачи мы пробовали несколько разных алгоритмов и подходов, начиная от простых алгоритмов, заканчивая нейросетями. В результате, в нашем случае, лучший результат показал алгоритм нахождения ближайших похожих книг для каждого пользователя посредством нахождения косинусного расстояния на основании матрицы взаимодействия пользователей и книг (матрица user-items).

Текущий ноутбук с решение содержит только тренировку и формирование модели по этому алгоритму. Все другие наши эксперименты сюда не входят.

__Общий алгоритм подготовки датасета для обучения__
(содержится в отдельном блокноте)
1. Сворачиваем датасет с книгами по названию и автору, также удаляем из него книги без названий. Для удобства сохраняем соответствие  recId для каждой книги в исходном датасете для книг в новом свернутом датасете (датасет книг: `data/books.csv`; соответствие recId: `data/books_map.csv`)

2. На основании датасета выдачи книг готови датасет взаимодействий пользователей и книг: перекодируем книги на основании `data/books_map.csv`, удаляем лишние поля и сворачиваем. В результате получаем датасет: `data/interactions.csv`

__Тренировка модели__
1. Делим подготовленный датасет `data/interactions.csv` на train и test случайным образом в соотношении 90/10

2. На основании train готовим матрицу вида users-items 

3. Обучаем модель используя библиотеку `impicit` и `CosineRecomender`

4. Строим датасет result, который содержит фактически взятые книги пользователей из датасета test, а также рекомендации модели. Для честной оценки в датасет result включаются только пользователи, кто взял 5 и более книг в датасете test.

5. Считаем метрику precision@5 по каждому пользователю (как процент угаданных книг из тех, которые он взял фактически). Считаем итоговую метрику Average Precision@5 по всем пользователям

6. Переучиваем модель на всем датасете и сохраняем для использования в flask приложении.

## Тренировка и оценка модели

In [50]:
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import implicit

from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

from implicit.nearest_neighbours import CosineRecommender

In [26]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [27]:
df = pd.read_csv('data/interactions.csv', parse_dates=['date']).drop('Unnamed: 0', axis=1)
df.rename(columns={'item_id': 'item', 'user_id': 'user'}, inplace=True)
df['interaction'] = 1
df.sort_values(by='date')
print(df.shape)
df.head(2)

(11383578, 4)


,date,user,item,interaction
0,2019-09-01,1188,12555,1
1,2019-09-01,23276,252072,1


In [28]:
data_train, data_test = train_test_split(df, test_size=0.1, random_state=42)
data_test = data_test[data_test['user'].isin(data_train['user'])]
data_test = data_test[data_test['item'].isin(data_train['item'])]

In [29]:
data_train.drop('date', axis=1, inplace=True)
data_train.drop_duplicates(ignore_index=True, keep='first', inplace=True)
print(data_train.shape)
data_train.head()

(9191619, 3)


,user,item,interaction
0,602580,128074,1
1,445326,306934,1
2,559995,905984,1
3,341602,1563321,1
4,469041,62978,1


In [30]:
person_c = CategoricalDtype(sorted(data_train.user.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(data_train.item.unique()), ordered=True)

row = data_train.user.astype(person_c).cat.codes
col = data_train.item.astype(thing_c).cat.codes
sparse_user_item = csr_matrix((data_train["interaction"], (row, col)), \
                           shape=(person_c.categories.size, thing_c.categories.size)).tocsr()

sparse_user_item = sparse_user_item.astype(np.float64)

In [31]:
sparse_user_item.sum(), sparse_user_item.shape

(9191619.0, (547133, 346976))

In [32]:
# перенумеруем пользователей и товары
userids = person_c.categories.to_list()
itemids = thing_c.categories.to_list()

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [33]:
result = data_test.groupby('user')['item'].unique().reset_index()
result.columns=['user', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result['actual_count'] = result['actual'].apply(lambda x: len(x))
result = result.loc[result['actual_count']>=5]
print(result.shape)
result.head(10)

(61937, 3)


,user,actual,actual_count
1,171,"[26612, 249379, 272308, 52383, 356434]",5
8,188,"[124303, 357246, 586439, 301465, 30697, 117820...",7
20,232,"[11527, 123661, 122056, 48793, 532234, 48254, ...",18
21,233,"[297395, 12446, 5613, 20779, 30167, 1190698, 2...",13
22,234,"[40843, 253158, 34861, 6566, 269092, 10199, 34...",14
23,235,"[268139, 9334, 57344, 26598, 35652, 116273, 32...",9
24,236,"[308318, 1334309, 5308, 21012, 1528923, 4950, ...",17
27,242,"[2389, 1869334, 40152, 118024, 55626, 90396, 1...",8
28,243,"[1, 470433, 1426384, 1398353, 16840, 5361, 377...",11
30,245,"[725170, 94272, 899996, 20381, 7618]",5


In [19]:
model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(sparse_user_item.T.tocsr(), 
          show_progress=True)

  0%|          | 0/346976 [00:00<?, ?it/s]

In [34]:
recs = model.recommend(userid=0, 
                        user_items=sparse_user_item.tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

[id_to_itemid[rec[0]] for rec in recs]

[12, 17936, 256275, 10245, 240293]

In [38]:
%%time

result['recommend'] = result['user'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 2.56 s, sys: 0 ns, total: 2.56 s
Wall time: 2.56 s


In [40]:
result['precision@5'] =  result.apply(lambda row: precision_at_k(row['recommend'], row['actual'], 5), axis=1)

In [42]:
print(result.shape)
result.loc[result['precision@5'] > 0].head(20)

(61937, 5)


,user,actual,actual_count,recommend,precision@5
23,235,"[268139, 9334, 57344, 26598, 35652, 116273, 32...",9,"[324604, 8327, 23185, 8346, 903291]",0.2
36,260,"[23578, 16060, 283791, 328346, 713934, 1691250...",16,"[439155, 272075, 416677, 1357235, 326471]",0.2
38,263,"[199425, 34334, 69308, 980673, 89019]",5,"[5897, 5914, 184088, 2109029, 980673]",0.2
58,291,"[139204, 16460, 17446, 106701, 6137, 390376, 1...",8,"[1271306, 150269, 11165, 146947, 31343]",0.2
76,324,"[1651837, 738145, 1205797, 380549, 99837, 52992]",6,"[380549, 1651837, 1082943, 738145, 547019]",0.6
88,344,"[9428, 10845, 20779, 1553876, 23802, 1672040, ...",22,"[248061, 2120183, 463933, 310588, 409517]",0.2
119,395,"[11177, 23802, 1525, 1807794, 33259, 1398353, ...",11,"[283791, 10668, 250688, 439155, 234344]",0.2
125,402,"[30060, 11462, 353219, 5721, 7449]",5,"[246777, 283791, 570219, 353219, 1133419]",0.2
181,480,"[325010, 254629, 63760, 43985, 42326, 3556, 23...",7,"[23802, 1271306, 16234, 1463117, 953670]",0.2
183,483,"[1656235, 1271049, 245538, 129798, 23802, 1088...",10,"[639595, 1271163, 105664, 174962, 19734]",0.2


In [46]:
print('Итоговая метрика Average Precision@5:', result['precision@5'].mean())

Итоговая метрика Average Precision@5: 0.06304470671810879


## Обучение модели на всех данных и сохранение

In [47]:
person_c = CategoricalDtype(sorted(df.user.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(df.item.unique()), ordered=True)

row = df.user.astype(person_c).cat.codes
col = df.item.astype(thing_c).cat.codes
sparse_user_item = csr_matrix((df["interaction"], (row, col)), \
                           shape=(person_c.categories.size, thing_c.categories.size)).tocsr()

sparse_user_item = sparse_user_item.astype(np.float64)

In [48]:
sparse_user_item.sum(), sparse_user_item.shape

(11383578.0, (553767, 355565))

In [49]:
# перенумеруем пользователей и товары
userids = person_c.categories.to_list()
itemids = thing_c.categories.to_list()

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [107]:
# сохраняем результат перенумерации 
joblib.dump(id_to_itemid, 'model/id_to_itemid')
joblib.dump(id_to_userid, 'model/id_to_userid')
joblib.dump(itemid_to_id, 'model/itemid_to_id')
joblib.dump(userid_to_id, 'model/userid_to_id')

# сохраняем матрицу
sparse_user_item
joblib.dump(sparse_user_item, 'model/sparse_user_item')

['model/sparse_user_item']

In [123]:
model = CosineRecommender(K=50, num_threads=4) 

model.fit(sparse_user_item.T.tocsr(), 
          show_progress=True)

  0%|          | 0/355565 [00:00<?, ?it/s]

In [137]:
%%time
recs = model.recommend(userid=userid_to_id[2255], 
                        user_items=sparse_user_item.tocsr(),   # на вход user-item matrix
                        N=50, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

[id_to_itemid[rec[0]] for rec in recs]

KeyError: 2255

In [128]:
# сохраняем модель
joblib.dump(model, 'model/model_cosine_k50')

['model/model_cosine_k50']

#### Функции для flask

In [73]:
def model_predictions(user_id):
    try:
        id = userid_to_id[user_id]
    except:
        return [] # для пользователей о ком ничего не делаем рекомендаций. В будущем можно будет рекомендовать топ или собирать их предпочтения
    
    recs = model.recommend(userid=id, 
                        user_items=sparse_user_item.tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)
    return [id_to_itemid[rec[0]] for rec in recs]
  
print(model_predictions(user_id=1188))
print(model_predictions(user_id=1111))
print(model_predictions(user_id=1133333333333333388))

[853656, 490513, 463397, 377311, 656324]
[1011857, 1203734, 6316, 91208, 80708]
[]


In [129]:
%%time
def get_json_books(predictions: [int]):
    result = []
    for pred in predictions:
        id = str(pred)
        book = books.loc[books['recId'] == id]
        title = book['title'].values[0]
        aut = book['aut'].values[0]
        result.append({"id": id, "title": title, "author": aut})
    return result
        
get_json_books([853656, 490513, 463397, 377311, 656324])

CPU times: user 539 ms, sys: 9.74 ms, total: 549 ms
Wall time: 543 ms


[{'id': '853656',
  'title': 'Капитанская дочка. Дубровский. Повести покойного Ивана Петровича Белкина',
  'author': 'Пушкин Александр Сергеевич'},
 {'id': '490513', 'title': 'Здравствуй, белолапый!', 'author': nan},
 {'id': '463397',
  'title': 'Трудный возраст и 6 "Б"',
  'author': 'Матвеева Людмила Григорьевна'},
 {'id': '377311',
  'title': 'Тимошкин ковш',
  'author': 'Беляков Иван Васильевич'},
 {'id': '656324',
  'title': 'Билеты в цирк',
  'author': 'Аксенова Анна Сергеевна'}]

In [134]:
%%time
def get_json_books2(predictions: [int]):
    result = []
    for pred in predictions:
        id = str(pred)
        b#ook = books.loc[books['recId'] == id]
        title = books.loc[books['recId'] == id, ['title', 'aut']]
        #aut = book['aut'].values[0]
        result.append({"id": id, "title": title, "author": 1})
    return result

get_json_books([853656, 490513, 463397, 377311, 656324])

CPU times: user 332 ms, sys: 76 µs, total: 332 ms
Wall time: 329 ms


[{'id': '853656',
  'title': 'Капитанская дочка. Дубровский. Повести покойного Ивана Петровича Белкина',
  'author': 'Пушкин Александр Сергеевич'},
 {'id': '490513', 'title': 'Здравствуй, белолапый!', 'author': nan},
 {'id': '463397',
  'title': 'Трудный возраст и 6 "Б"',
  'author': 'Матвеева Людмила Григорьевна'},
 {'id': '377311',
  'title': 'Тимошкин ковш',
  'author': 'Беляков Иван Васильевич'},
 {'id': '656324',
  'title': 'Билеты в цирк',
  'author': 'Аксенова Анна Сергеевна'}]

In [122]:
def get_history(use_id):
    return list(interactions.loc[interactions['user_id'] == 1188]['item_id'].values)
                

[{'id': '12555', 'title': 'Семья 3х1', 'author': 'Майорош Нора'},
 {'id': '94424',
  'title': 'Неизвестный цветок',
  'author': 'Платонов Андрей Платонович'},
 {'id': '112234', 'title': 'Мальчики', 'author': 'Чехов Антон Павлович'},
 {'id': '264533', 'title': 'Кусака', 'author': 'Андреев Леонид'},
 {'id': '114610',
  'title': 'Приключения Незнайки и его друзей , Незнайка в Солнечном городе , Незнайка на Луне',
  'author': 'Носов Николай Николаевич'},
 {'id': '256134',
  'title': 'Этажи леса',
  'author': 'Пришвин Михаил Михайлович'},
 {'id': '1301',
  'title': 'Стрижонок Скрип',
  'author': 'Астафьев Виктор Петрович'},
 {'id': '269023', 'title': 'Ю-Ю', 'author': 'Куприн Александр Иванович'},
 {'id': '285528',
  'title': 'Приемыш',
  'author': 'Мамин-Сибиряк Дмитрий Наркисович'},
 {'id': '1526',
  'title': 'Чей нос лучше?',
  'author': 'Бианки Виталий Валентинович'},
 {'id': '33449',
  'title': 'Карлссон, который живет на крыше',
  'author': 'Линдгрен Астрид'},
 {'id': '2878',
  'title'